# **FirstAid Generative QA Assistant** - Using LoRA Fine-Tuning

### **ENV SETUP:**

In [ ]:
# Install dependencies
!pip install -q transformers datasets peft accelerate bitsandbytes trl evaluate rouge_score gradio

### IMPORTS

In [ ]:
import torch
import time
import pandas as pd
import numpy as np

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)

from peft import LoraConfig, get_peft_model
import evaluate

### LOADING DATASET

In [ ]:
dataset = load_dataset("i-am-mushfiq/FirstAidQA")
dataset
dataset["train"][0]

### Train / Validation Split:

In [ ]:
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"]
val_dataset = dataset["test"]

print(len(train_dataset), len(val_dataset))

### System Instruction Formatting

In [ ]:
def format_prompt(example):
    return f"""### Instruction:
You are a professional first aid assistant.
Provide clear, step-by-step emergency guidance.
This is for educational purposes only.

Question:
{example['question']}

### Response:
{example['answer']}"""

## Apply formatting:
train_dataset = train_dataset.map(lambda x: {"text": format_prompt(x)})
val_dataset = val_dataset.map(lambda x: {"text": format_prompt(x)})

### Load Tokenizer & Model

In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

### Tokenization

In [ ]:
max_length = 512

def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=max_length
    )

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

### Re-usable Experiment Function

In [ ]:
def run_experiment(exp_name, learning_rate, lora_rank, epochs):

    print(f"\nRunning {exp_name}")

    # Apply LoRA
    lora_config = LoraConfig(
        r=lora_rank,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    lora_model = get_peft_model(model, lora_config)

    training_args = TrainingArguments(
        output_dir=f"./{exp_name}",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        learning_rate=learning_rate,
        num_train_epochs=epochs,
        logging_steps=50,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        fp16=True,
        report_to="none"
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    trainer = Trainer(
        model=lora_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator
    )

    start_time = time.time()
    trainer.train()
    training_time = time.time() - start_time

    eval_results = trainer.evaluate()

    lora_model.save_pretrained(f"{exp_name}_adapter")

    return {
        "experiment": exp_name,
        "learning_rate": learning_rate,
        "lora_rank": lora_rank,
        "epochs": epochs,
        "eval_loss": eval_results["eval_loss"],
        "training_time_sec": training_time
    }

### Runnning all (3) Experiments

In [ ]:
results = []

results.append(run_experiment("exp1", 2e-4, 16, 2))
results.append(run_experiment("exp2", 5e-5, 16, 3))
results.append(run_experiment("exp3", 2e-4, 32, 1))

results_df = pd.DataFrame(results)
results_df

# saving results
results_df.to_csv("experiment_results.csv", index=False)

### **Evaluation (BLEU & ROUGE)**

In [ ]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# generate predictions:
def generate_answer(question, model):
    prompt = f"""### Instruction:
You are a professional first aid assistant.

Question:
{question}

### Response:"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# test on validation dataset:
sample = val_dataset.select(range(50))

predictions = []
references = []

for example in sample:
    question = example["text"].split("Question:")[1].split("### Response:")[0].strip()
    reference = example["text"].split("### Response:")[1].strip()

    pred = generate_answer(question, model)

    predictions.append(pred)
    references.append([reference])

# compute metrics:
bleu_score = bleu.compute(predictions=predictions, references=references)
rouge_score = rouge.compute(predictions=predictions, references=[r[0] for r in references])

print("BLEU:", bleu_score)
print("ROUGE:", rouge_score)

### Perplexity

In [ ]:
import math

def compute_perplexity(eval_loss):
    return math.exp(eval_loss)

results_df["perplexity"] = results_df["eval_loss"].apply(compute_perplexity)
results_df

### Base Model Comparison

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

print(generate_answer("How do I treat a burn?", base_model))

### Gradio App Deployment

In [ ]:
import gradio as gr

def chatbot(question):
    return generate_answer(question, model)

interface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="FirstAid Emergency Assistant",
    description="Educational purposes only. Not a substitute for professional medical advice."
)

interface.launch()

### Saving the Final Model

In [ ]:
model.save_pretrained("final_firstaid_model")
tokenizer.save_pretrained("final_firstaid_model")